### Drought Analysis using Drought Indices through Calculation of the following Simple Drought indices - VCI, TCI, PCI, SMCI, etc. (Indices are computed numerical representations of drought severity assessed using climatic or hydrometeorological input)

In [1]:
import ee
import geemap

In [2]:
# extracting region of interest (Nigeria) from the FAO Administrative Units Data

m = geemap.Map(center=[9,7], zoom=6, height='500px')
nija = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Nigeria'))
m.addLayer(nija, {}, 'Nigeria Boundary')
m

Map(center=[9, 7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### Calculating Nigeria's Normalized Difference Vegetation Index (NDVI) Using the MODIS Terra Vegetation Indices 16-Day Global 1km

In [3]:
# importing MODIS dataset and filtering the date and selecting the ndvi band
data = ee.ImageCollection("MODIS/061/MOD13A2").filterDate('2023-01-01', '2023-11-30').select('NDVI')

data.first()

In [4]:
#Applying scaling factor, which will allow us to use the original MODIS value of -1 to +1 for the visualization

ndviscale = data.map(lambda img: img.multiply(0.0001).copyProperties(img, ['system:time_start', 'system:time_end']))

In [5]:
ndviscale.size()

In [6]:
ndviVis = {
  'min': 0.0,
  'max': 1,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
}



vis_params = {
    'color': '#000000',
    'width': 1,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [7]:
ngndvi = ndviscale.map(lambda img: img.clipToCollection(nija))

In [8]:
#Visualizing scaled NDVI of Nigeria.. 0.0 means no Vegetation and the closer to 1 the more the vegetation
m = geemap.Map()
m.addLayer(ngndvi, ndviVis, 'NigeriaNDVI')
m.addLayer(nija.style(**vis_params), {}, 'Nigeria')
m.add_colorbar(ndviVis, label='Nigeria NDVI', layer_name='NigeriaNDVI')
text = 'Map Created by Emeka Orji'
text2 = 'Data Source: MOD13A2.061 Terra Vegetation'
m.add_text(text2, position='bottomleft', fontsize=10)
m.add_text(text)
m.center_object(nija, zoom=6)
m

Map(center=[9.589444610453397, 8.089338153274129], controls=(WidgetControl(options=['position', 'transparent_b…

#### Calculating Land Surface Temperature of Nigeria Using the MODIS Terra Land Surface Temperature and Emissivity Daily Global 1km

In [9]:
# importing MODIS LST dataset, filtering the date for February and March 2024, and selecting the LST band
data_lst = ee.ImageCollection("MODIS/061/MOD11A1").filterDate('2024-02-01', '2024-03-30').select('LST_Day_1km')

In [10]:
data_lst.first()

In [11]:
# Scaling the LST image and converting from kelvin to Celsuis using lambda function

lst = data_lst.map(lambda img: img.multiply(0.02).subtract(273.15).copyProperties(img, ['system:time_start']))

In [12]:
lstVis = {
  'min': 20,
  'max': 40,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
}

In [13]:
nglst = lst.map(lambda img: img.clipToCollection(nija))

In [14]:
#Visualizing scaled LST of Nigeria
m = geemap.Map()
m.addLayer(nglst, lstVis, 'NigeriaLST')
m.addLayer(nija.style(**vis_params), {}, 'Nigeria')
m.add_colorbar(lstVis, label='Nigeria LST (celsuis)', layer_name='NigeriaLST')
text = 'Map Created by Emeka Orji'
text2 = 'Data Source: MOD11A1.061 Terra Land Surface Temperature'
m.add_text(text2, position='bottomleft', fontsize=10)
m.add_text(text)
m.center_object(nija, zoom=6)
m

Map(center=[9.589444610453397, 8.089338153274129], controls=(WidgetControl(options=['position', 'transparent_b…

In [15]:
# Mapping Nigeria highest LST between February and March 2024

extremeLST = nglst.reduce(ee.Reducer.max())
m = geemap.Map()
m.addLayer(extremeLST, {'min':20, 'max':40, 'palette': ['blue', 'limegreen', 'yellow', 'darkorange', 'red']}, 'LSTMax')
m.addLayer(nija.style(**vis_params), {}, 'Nigeria')
m.add_colorbar(lstVis, label='Nigeria"s Highest LST (celsuis)', layer_name='NigeriaLST')
text = 'Map Created by Emeka Orji'
m.add_text(text)
m.center_object(nija, zoom=6)
m

Map(center=[9.589444610453397, 8.089338153274129], controls=(WidgetControl(options=['position', 'transparent_b…

#### Calculating for VCI (Vegetation Condition Index) using the MODIS NDVI Data
 VCI calculation from MODIS NDVI \
 VCI = (NDVI - NDVImin) / (NDVIMax - NDVImin) * 100  // VCI Formula

In [18]:
map = geemap.Map()
ndvi = ee.ImageCollection("MODIS/061/MOD13A2").filterDate('2023-01-01', '2024-01-01').select('NDVI')

ndviVis = {
  'min': 0.0,
  'max': 9000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
}

map.addLayer(ndvi, ndviVis, 'NDVI')
map.add_colorbar(ndviVis, label='World NDVI', layer_name='Global NDVI')
text = 'Map Created by Emeka Orji'
map.add_text(text, position='bottomleft', fontsize=15)
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [21]:
# Calculating VCI
# VCI calculation from MODIS NDVI
# VCI = (NDVI - NDVImin) / (NDVIMax - NDVImin) * 100 //VCI Formula

# I am multiplying with the scale factor of 0.0001 as indicated on the MODIS data description
ndviMin = ndvi.reduce(ee.Reducer.min()).multiply(0.0001)
ndviMax = ndvi.reduce(ee.Reducer.max()).multiply(0.0001)


# Using Lambda function

#ndmap = ndvi.map(lambda img: img.select('NDVI').subtract(ndviMin).divide((ndviMax.subtract(ndviMin))).multiply(100).rename('VCI'))
# (we have multiplied 100 to normalise our output ie to define ranges from 0 to 100, creating output results from 0 to 100)
#OR using a full function

def CalVci(img):
    band = img.select('NDVI').multiply(0.0001)
    VCI = band.subtract(ndviMin).divide((ndviMax.subtract(ndviMin))).multiply(100).rename('VCI')
    return img.addBands(VCI)

ndtovci = ndvi.map(CalVci)

vci = ndtovci.select('VCI')

vci.first()

In [24]:
# Visualizing the Drought Condition of Nigeria for 2023 using the Median Vegetation Condition Index (VCI)

ng_vci_median = vci.select('VCI').median().clipToCollection(nija)
#ngvci = vci.map(lambda img: img.clipToCollection(nija))

map = geemap.Map()


#Separating the visualization to add to the colorbar

vciVis = {
  'min': 0,
  'max': 100,
  'palette': ['#a50026', '#d73027', '#f46d43', '#fdae61', '#fee08b', 
              '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', '#006837'
             ],
}

# map.addLayer(ng_vci_median, {'min':0, 'max':100, 'palette':['#a50026', '#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', 
#                                                   '#006837']}, 'NigeriaDrought')


map.addLayer(ng_vci_median, vciVis, 'NigeriaDrought')

drought_class = """
Value	Color	Description
1	#a50026	Extreme Drought
2	#d73027	Severe Drought
3	#f46d43	Moderate Drought
4	#fdae61	Mild Drought
5	#fee08b	Light Drought
6	#d9ef8b	No Drought
7	#a6d96a	No Drought
8	#66bd63	No Drought
9	#1a9850	No Drought
10	#006837	No Drought
"""

legend_dict = geemap.legend_from_ee(drought_class)


map.addLayer(nija.style(**vis_params), {}, 'Nigeria')
text = 'Map Created by Emeka Orji'
text2 = 'Data Source: MOD13A2.061 Terra Vegetation'
map.add_text(text2, position='bottomleft', fontsize=12)
map.add_text(text)
map.add_colorbar(vciVis, label='Nigeria Drought Condition', layer_name='NigeriaVCI')
map.add_legend(title="Nigeria Drought Condition", legend_dict=legend_dict)
map.center_object(nija, zoom=6)
map

Map(center=[9.589444610453397, 8.089338153274129], controls=(WidgetControl(options=['position', 'transparent_b…

##### Legend Visualization examples

In [ ]:
# Map = geemap.Map(center=[40, -100], zoom=4)

# legend_dict = {
#     '11 Open Water': '466b9f',
#     '12 Perennial Ice/Snow': 'd1def8',
#     '21 Developed, Open Space': 'dec5c5',
#     '22 Developed, Low Intensity': 'd99282',
#     '23 Developed, Medium Intensity': 'eb0000',
#     '24 Developed High Intensity': 'ab0000',
#     '31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
#     '41 Deciduous Forest': '68ab5f',
#     '42 Evergreen Forest': '1c5f2c',
#     '43 Mixed Forest': 'b5c58f',
#     '51 Dwarf Scrub': 'af963c',
#     '52 Shrub/Scrub': 'ccb879',
#     '71 Grassland/Herbaceous': 'dfdfc2',
#     '72 Sedge/Herbaceous': 'd1d182',
#     '73 Lichens': 'a3cc51',
#     '74 Moss': '82ba9e',
#     '81 Pasture/Hay': 'dcd939',
#     '82 Cultivated Crops': 'ab6c28',
#     '90 Woody Wetlands': 'b8d9eb',
#     '95 Emergent Herbaceous Wetlands': '6c9fb8',
# }

# nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
# landcover = nlcd.select('landcover')

# Map.addLayer(landcover, {}, 'NLCD Land Cover 2019')
# Map.add_legend(title="NLCD Land Cover Classification", legend_dict=legend_dict)
# Map

In [ ]:
# Map = geemap.Map(add_google_map=False)

# labels = ['One', 'Two', 'Three', 'Four', 'etc']

# # colors can be defined using either hex code or RGB (0-255, 0-255, 0-255)
# colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
# # legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

# Map.add_legend(labels=labels, colors=colors, position='bottomright')
# Map

In [ ]:
# Built in Legend

# Map = geemap.Map(center=[40, -100], zoom=4)
# Map.add_basemap('HYBRID')

# nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
# landcover = nlcd.select('landcover')

# Map.addLayer(landcover, {}, 'NLCD Land Cover 2019')
# Map.add_legend(
#     title="NLCD Land Cover Classification", builtin_legend='NLCD', height='465px'
# )
# Map

In [51]:
LandCover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select('landcover')

newvci = vci.select('VCI').mean()

# apply threshold to selected classes of landcover dataset
mask = LandCover.eq(11).add(LandCover.eq(14)).add(LandCover.eq(20)).add(LandCover.eq(30))
# Update VCI with Mask
maskedVCI = newvci.updateMask(mask)

In [52]:
condition = maskedVCI.where(maskedVCI.lte(10), 1).where(maskedVCI.gte(10).And(maskedVCI.lt(20)), 2).where(maskedVCI.gte(20).And(maskedVCI.lt(30)), 3)\
.where(maskedVCI.gte(30).And(maskedVCI.lt(40)), 4).where(maskedVCI.gte(40).And(maskedVCI.lt(50)), 5).where(maskedVCI.gte(50).And(maskedVCI.lt(60)), 6)\
.where(maskedVCI.gte(60).And(maskedVCI.lt(70)), 7).where(maskedVCI.gte(70).And(maskedVCI.lt(80)), 8).where(maskedVCI.gte(80).And(maskedVCI.lt(90)), 9)\
.where(maskedVCI.gte(90), 10)

map = geemap.Map()

conditionParams = {'min':0, 'max':10, 'palette': ['bc0000','ff6c54','fffb6e','e9cb38','bcabff',
                                            'c93aed','4385ff','81eeff','89ed3e','02a100']}
map.addLayer(condition, conditionParams, 'Vegetation Condition 2022')
#map.center_object(nija, zoom=6)
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [53]:
# 'bc0000', 'Extreme Dry'));//1
# legend.add(makeRow('ff6c54', 'Severe Dry'));//2
# legend.add(makeRow('ffffc0', 'Moderate Dry'));//3
# legend.add(makeRow('e9cb38', 'Mild Dry'));//4
# legend.add(makeRow('bcabff', 'No Drought'));//5
# legend.add(makeRow('c93aed', 'No Drought'));//6
# legend.add(makeRow('4385ff', 'Mild Wet'));//7
# legend.add(makeRow('81eeff', 'Moderate Wet'));//8
# legend.add(makeRow('89ed3e', 'Severe Wet'));//9
# legend.add(makeRow('02a100', 'Extreme Wet'));//10

ee_class_table = """
Value	Color	Description
1	bc0000	Extreme Dry
2	ff6c54	Severe Dry
3	ffffc0	Moderate Dry
4	e9cb38	Mild Dry
5	bcabff	No Drought
6	c93aed	No Drought
7	4385ff	Mild Wet
8	81eeff	Moderate Wet
9	89ed3e	Severe Wet
10	02a100	Extreme Wet
"""

legend_dict = geemap.legend_from_ee(ee_class_table)
map.add_legend(title="Drought Severity", legend_dict=legend_dict)

#### Calculating for TCI (Temperature Condition Index) using the MODIS Temperature Data
TCI calculation from MODIS Temperature \
TCI = (LST - LSTmax) / (LSTMax - LSTmin) * 100  // TCI Formula

In [19]:
map = geemap.Map()
ModLst = ee.ImageCollection("MODIS/061/MOD11A1").filterDate('2023-01-01', '2023-12-30').select('LST_Day_1km')

Lst = ModLst.map(lambda img: img.multiply(0.02).subtract(273.15).copyProperties(img, ['system:time_start']))

lstVis = {
    'min':20,
    'max':40,
    'palette':[
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ],
}


map.addLayer(Lst, lstVis, 'LST')
map.add_colorbar(lstVis, label='World LST', layer_name='Global LST', transparent_bg=True, font_size=8)
text = 'Map Created by Emeka Orji'
map.add_text(text, position='bottomleft', fontsize=15)
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
#Calculating TCI from MODIS LST
lstMin = Lst.reduce(ee.Reducer.min())
lstMax = Lst.reduce(ee.Reducer.max())


#TCI = (LST - LSTmax) / (LSTMax - LSTmin) * -100 (we are multiplying by -100 so we can have our result ranges in discrete numbers otherwise it
# will calculate the result in degree of continous numbers
def tciCal(img):
    TCI = img.select('LST_Day_1km').subtract(lstMax).divide(lstMax.subtract(lstMin)).multiply(-100).rename('TCI')
    return img.addBands(TCI)

tci = Lst.map(tciCal).select('TCI')
tci.size()

In [21]:
tci.first()

In [22]:
map = geemap.Map()

tciVis = {
    'min':0,
    'max':100,
    'palette':[
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ],
}

map.addLayer(tci, tciVis, 'TCI')
text = 'Map Created by Emeka Orji'
text2 = 'Data Source: MOD11A1.061 Terra Land Surface Temperature'
map.add_colorbar(tciVis, label='World TCI', layer_name='Global TCI', transparent_bg=True, font_size=8)
map.add_text(text2, position='bottomleft', fontsize=10)
map.add_text(text)
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [23]:
ngTci = tci.map(lambda img: img.clipToCollection(nija))
map = geemap.Map()
map.addLayer(ngTci, tciVis, 'NigeriaTCI')
map.addLayer(nija.style(**vis_params), {}, 'Nigeria')
text = 'Map Created by Emeka Orji'
text2 = 'Data Source: MOD11A1.061 Terra Land Surface Temperature'
map.add_colorbar(tciVis, label='Nigeria TCI (Celsuis)', layer_name='Nigeria TCI', font_size=8)
map.add_text(text2, position='bottomleft', fontsize=10)
map.add_text(text)
map.center_object(nija, zoom=6)
map

Map(center=[9.589444610453397, 8.089338153274129], controls=(WidgetControl(options=['position', 'transparent_b…

#### Calculating for PCI (Precipitation Condition Index) using the GPM: Monthly Global Precipitation Measurement
PCI calculation from GPM: Monthly Global Precipitation Measurement \
PCI = (Pre - PreMin) / (PreMax - PreMin) * 100  // PCI Formula

In [24]:
predata = ee.ImageCollection("TRMM/3B43V7").filterDate('2019-01-01', '2019-12-01').select('precipitation')

Minp = predata.reduce(ee.Reducer.min())
Maxp =  predata.reduce(ee.Reducer.max())

def pciCal(img):
    PCI = img.select('precipitation').subtract(Minp).divide(Maxp.subtract(Minp)).multiply(100).rename('PCI')
    return img.addBands(PCI)

pci = predata.map(pciCal).select('PCI')
pci.size()

In [25]:
pci.first()

In [26]:
m = geemap.Map()

PCIVis = {
    'min':0,
    'max':100,
    'palette':['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}


m.addLayer(pci, PCIVis, 'Global PCI (mm/hr)');
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
ngpci = pci.map(lambda img: img.clipToCollection(nija))
map = geemap.Map()
map.addLayer(ngpci, PCIVis, 'NigeriaPCI')
map.addLayer(nija.style(**vis_params), {}, 'Nigeria')
text = 'Map Created by Emeka Orji'
text2 = 'Data Source: MOD11A1.061 Terra Land Surface Temperature'
map.add_colorbar(tciVis, label='Nigeria TCI (Celsuis)', layer_name='Nigeria TCI', font_size=8)
map.add_text(text2, position='bottomleft', fontsize=10)
map.add_text(text)
map.center_object(nija, zoom=6)
map

#### Calculating for SMCI (Soil Moisture Condition Index) using the Famine Early Warning Systems Network (FEWS NET) Land Data Assimilation System
SMCI calculation from FEWS NET Data \
SMCI = (Soil - MinS) / (MaxS - MinS) * 100  // SMCI Formula

In [34]:
sm_data = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001").filterDate('2023-01-01', '2024-01-01')
soil = sm_data.select('SoilMoi00_10cm_tavg')
m = geemap.Map()
soilVis = {
    'min':0.0,
    'max':1,
    'opacity': 1.0,
    'palette':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

m.addLayer(soil, soilVis, 'Soil Moisture (0 - 10 cm underground)');
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [37]:
MinS = soil.reduce(ee.Reducer.min())
MaxS = soil.reduce(ee.Reducer.max())

def soilCal(img):
    SMCI = img.select('SoilMoi00_10cm_tavg').subtract(MinS).divide(MaxS.subtract(MinS)).multiply(100).rename('SMCI')
    return img.addBands(SMCI)
soilsmci = soil.map(soilCal)
soilsmci.first()

In [39]:
smci = soilsmci.select('SMCI')

m = geemap.Map()
smciVis = {
    'min':0.0,
    'max':100,
    'opacity': 1.0,
    'palette':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

m.addLayer(smci, smciVis, 'Global SMCI');
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…